<a href="https://colab.research.google.com/github/somkiatth/git/blob/master/kumllibAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 61.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=f97ed6606db3f194f0a498e1ac3543758be86d5cff3fceac50bcf5ad147fae62
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
mldf=spark.read.csv("/content/census-income-all.csv", header=True, inferSchema=True)

In [6]:
mldf.show(5)

+---+----------------+------+---------+-------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+-----+
|age|       workclass|weight|education|edu num|    marital status|       occupation| relationship| race|   sex|captial-gain|capital-loss|hours-per-week|native country|label|
+---+----------------+------+---------+-------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+-----+
| 39|       State-gov| 77516|Bachelors|     13|     Never-married|     Adm-clerical|Not-in-family|White|  Male|        2174|           0|            40| United-States|   No|
| 50|Self-emp-not-inc| 83311|Bachelors|     13|Married-civ-spouse|  Exec-managerial|      Husband|White|  Male|           0|           0|            13| United-States|   No|
| 38|         Private|215646|  HS-grad|      9|          Divorced|Handlers-cleaners|Not-in-family|White|  Male|           0|      

In [8]:
cstrain,cstest = mldf.randomSplit([0.8,0.2])

In [9]:
cstrain.count(),cstest.count()

(38986, 9856)

In [ ]:
cstrain.write.csv("/content/censustrain.csv")

In [20]:
cstrainload=spark.read.load("/content/censustrain.csv",format='csv')

In [22]:
cstrainload.count()

38986

In [29]:
cstrain.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- edu num: integer (nullable = true)
 |-- marital status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- captial-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native country: string (nullable = true)
 |-- label: string (nullable = true)



In [25]:
from pyspark.ml.feature import VectorAssembler

In [35]:
va=VectorAssembler(inputCols=['age','edu num','captial-gain','capital-loss','hours-per-week'],outputCol='features')

In [41]:
vtraindf=va.transform(cstrain)

In [42]:
vtraindf.show(5)

+---+---------+------+---------+-------+--------------+----------+------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+
|age|workclass|weight|education|edu num|marital status|occupation|relationship| race|   sex|captial-gain|capital-loss|hours-per-week|native country|label|            features|
+---+---------+------+---------+-------+--------------+----------+------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+
| 17|        ?| 27251|     11th|      7|       Widowed|         ?|   Own-child|White|  Male|           0|           0|            40| United-States|   No|[17.0,7.0,0.0,0.0...|
| 17|        ?| 34019|     10th|      6| Never-married|         ?|   Own-child|White|  Male|           0|           0|            20| United-States|   No|[17.0,6.0,0.0,0.0...|
| 17|        ?| 34088|     12th|      8| Never-married|         ?|   Own-child|White|Female|           0|           0|  

In [38]:
from pyspark.ml.feature import StringIndexer

In [45]:
si=StringIndexer(inputCol='label',outputCol='label_idx',stringOrderType='frequencyDesc')

In [46]:
labelindex = si.fit(vtraindf)

In [47]:
vidxtraindf=labelindex.transform(vtraindf)

In [50]:
vidxtraindf.sample(1/4000).show()

+---+------------+------+------------+-------+--------------------+-----------------+--------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+---------+
|age|   workclass|weight|   education|edu num|      marital status|       occupation|  relationship| race|   sex|captial-gain|capital-loss|hours-per-week|native country|label|            features|label_idx|
+---+------------+------+------------+-------+--------------------+-----------------+--------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+---------+
| 22|     Private| 61850|     Masters|     14|       Never-married|            Sales|Other-relative|White|Female|           0|           0|            21| United-States|   No|[22.0,14.0,0.0,0....|      0.0|
| 23|     Private|195508|Some-college|     10|       Never-married|Handlers-cleaners|     Own-child|White|  Male|           0|           0|            20| United-States|   